In [41]:
using CSV
using DataFrames

using JuMP
using Gurobi

using LinearAlgebra

## Import data

In [14]:
# Paths (relative to notebook structure)
restaurant_path    = "../clean_data/restaurant_data.csv"
scrap_path         = "../clean_data/food_scrap_locations.csv"
neighborhood_path  = "../clean_data/neighborhood_supply.csv"

# Read CSVs into DataFrames
restaurant_data       = CSV.read(restaurant_path, DataFrame)
food_scrap_locations  = CSV.read(scrap_path, DataFrame)
neighborhood_supply   = CSV.read(neighborhood_path, DataFrame)

# Preview the first few rows
println(first(restaurant_data, 5))
println(first(food_scrap_locations, 5))
println(first(neighborhood_supply, 5))


5×3 DataFrame
 Row │ latitude  longitude  waste   
     │ Float64   Float64    Float64 
─────┼──────────────────────────────
   1 │  40.6313   -73.9472  22593.5
   2 │  40.7144   -73.8319  54589.8
   3 │  40.7893   -73.9753  57175.8
   4 │  40.7498   -73.9728  26668.3
   5 │  40.7578   -73.9825  74368.1
5×27 DataFrame
 Row │ Borough    NTAName                  SiteName                           SiteAddr                           Hosted_By                       Open_Month  Day_Hours                          Notes                      Website                            BoroCD  CouncilDis  ct2010   BBL       BIN      Latitude  Longitude  PolicePrec  Object.ID  Location.Point                App.Android  App.iOS  X.Assembly.District  X.Congress.District  DSNY.District  X.DSNY.Section  DSNY.Zone  Senate.District 
     │ String15   String                   String                             String                             String                          String31    String                  

# Clean the data:

## Ensure no commas in numbers, each field cast as correct type, drop unnecessary columns

In [39]:
# clean restaurant data
# Columns: latitude | longitude | waste
rename!(restaurant_data, names(restaurant_data)[3] => :supply)

# Ensure Float64
restaurant_data.supply = Float64.(restaurant_data.supply)
restaurant_data.latitude = Float64.(restaurant_data.latitude)
restaurant_data.longitude = Float64.(restaurant_data.longitude)


# ================================
# 3. CLEAN FOOD SCRAP CENTER DATA
# ================================
# Keep only coordinates we need

food_scrap_locations.latitude = Float64.(food_scrap_locations.latitude)
food_scrap_locations.longitude = Float64.(food_scrap_locations.longitude)

# Keep only coordinate columns in food_scrap_locations
select!(food_scrap_locations, [:latitude, :longitude])

# ================================
# 4. CLEAN NEIGHBORHOOD SUPPLY DATA
# ================================
# Rename demand column for clarity
rename!(neighborhood_supply, names(neighborhood_supply)[4] => :supply_gap)

# Now neighborhood_supply.supply_gap might be String OR Float64.
# Only do replace/parse if it's strings.
if eltype(neighborhood_supply.supply_gap) <: AbstractString
    neighborhood_supply.supply_gap =
        parse.(Float64, replace.(neighborhood_supply.supply_gap, "," => ""))
end

# Demand = positive deficit, surplus -> 0
neighborhood_supply.demand = max.(0.0, -neighborhood_supply.supply_gap)

neighborhood_supply.latitude  = Float64.(neighborhood_supply.latitude)
neighborhood_supply.longitude = Float64.(neighborhood_supply.longitude)

# keep only necessary columns from neighborhood supply
select!(neighborhood_supply, [:latitude, :longitude, :demand])

# ================================
# 5. SHOW CLEANED HEADS
# ================================
println("=== Restaurants (cleaned) ===")
println(first(restaurant_data, 5))

println("\n=== Food Scrap Locations (cleaned) ===")
println(first(food_scrap_locations, 5))

println("\n=== Neighborhood Supply (cleaned) ===")
println(first(neighborhood_supply, 5))

=== Restaurants (cleaned) ===
5×3 DataFrame
 Row │ latitude  longitude  supply  
     │ Float64   Float64    Float64 
─────┼──────────────────────────────
   1 │  40.6313   -73.9472  22593.5
   2 │  40.7144   -73.8319  54589.8
   3 │  40.7893   -73.9753  57175.8
   4 │  40.7498   -73.9728  26668.3
   5 │  40.7578   -73.9825  74368.1

=== Food Scrap Locations (cleaned) ===
5×2 DataFrame
 Row │ latitude  longitude 
     │ Float64   Float64   
─────┼─────────────────────
   1 │  40.6355   -74.0228
   2 │  40.7526   -73.969
   3 │  40.7635   -74.0002
   4 │  40.762    -73.9693
   5 │  40.7174   -74.0108

=== Neighborhood Supply (cleaned) ===
5×3 DataFrame
 Row │ latitude  longitude  demand    
     │ Float64   Float64    Float64   
─────┼────────────────────────────────
   1 │  40.8267   -73.9217  1.02143e5
   2 │  40.8321   -73.8887  3.33493e5
   3 │  40.8016   -73.9374  0.0
   4 │  40.8469   -73.8918  1.13653e5
   5 │  40.6193   -73.9733  0.0


## Get vectors for supply[i] for all restaurants, demand[k] for all neighborhoods, and cij[i,j] and cjk[j,k] to plug directly into JuMP model

In [58]:
# ================
# 6. EXTRACT VECTORS
# ================
R = nrow(restaurant_data)
D = nrow(food_scrap_locations)
N = nrow(neighborhood_supply)

supply = restaurant_data.supply              # s_i
demand = neighborhood_supply.demand          # d_k

# ================
# 7. MANHATTAN DISTANCE FUNCTION
# ================
manhattan(lat1, lon1, lat2, lon2) = abs(lat1 - lat2) + abs(lon1 - lon2)

# ================
# 8. COST MATRICES
# ================
# cij: Restaurants (i) → Donation centers (j)
cij = [manhattan(restaurant_data.latitude[i], restaurant_data.longitude[i],
                 food_scrap_locations.latitude[j], food_scrap_locations.longitude[j])
       for i in 1:R, j in 1:D]

# cjk: Donation centers (j) → Neighborhoods (k)
cjk = [manhattan(food_scrap_locations.latitude[j], food_scrap_locations.longitude[j],
                 neighborhood_supply.latitude[k], neighborhood_supply.longitude[k])
       for j in 1:D, k in 1:N]

println("Size of cij (R x D): ", size(cij))
println("Size of cjk (D x N): ", size(cjk))


Size of cij (R x D): (321, 201)
Size of cjk (D x N): (201, 591)


# Sanity check data

In [62]:
println("Demand: min = ", minimum(demand), ", max = ", maximum(demand))
println("Any NaN in demand? ", any(isnan.(demand)))
println("Any Inf in demand? ", any(isinf.(demand)))

println("Any NaN in cij? ", any(isnan.(cij)))
println("Any Inf in cij? ", any(isinf.(cij)))

println("Any NaN in cjk? ", any(isnan.(cjk)))
println("Any Inf in cjk? ", any(isinf.(cjk)))

Demand: min = 0.0, max = 4.27814874096381e6
Any NaN in demand? false
Any Inf in demand? false
falseaN in cij? 
falsenf in cij? 
Any NaN in cjk? false
Any Inf in cjk? false


In [64]:
R = nrow(restaurant_data)
D = nrow(food_scrap_locations)
N = nrow(neighborhood_supply)

println("R, D, N = ", (R, D, N))

R, D, N = (321, 201, 591)


In [68]:
println("Size of cij: ", size(cij))  # should be (R, D)
println("Size of cjk: ", size(cjk))  # should be (D, N)

Size of cij: (321, 201)
Size of cjk: (201, 591)


# Cost-Reduction Optimization Model 

We formulate the minimum-cost redistribution problem below.

**Sets**

- $R$: restaurants (supply nodes)  
- $D$: food scrap / donation centers (transshipment nodes)  
- $N$: neighborhoods (demand nodes)

**Parameters**

- $s_i$: supply at restaurant $i \in R$  
- $d_k$: demand at neighborhood $k \in N$  
- $c_{ij}$: cost of transporting one unit from restaurant $i$ to donation center $j$  
- $c_{jk}$: cost of transporting one unit from donation center $j$ to neighborhood $k$  
- $M$: large penalty coefficient for unmet demand

**Decision Variables**

- $x_{ij} \ge 0$: shipment from restaurant $i$ to donation center $j$  
- $y_{jk} \ge 0$: shipment from donation center $j$ to neighborhood $k$  
- $u_k \ge 0$: unmet demand at neighborhood $k$

---

### **Objective: Minimize Total Cost**

$ \displaystyle
\min\;
\sum_{i \in R} \sum_{j \in D} c_{ij} x_{ij}
\;+\;
\sum_{j \in D} \sum_{k \in N} c_{jk} y_{jk}
\;+\;
M \sum_{k \in N} u_k
$

---

### **Constraints**

**1. Restaurant supply limits**

$ \displaystyle
\sum_{j \in D} x_{ij} \le s_i \quad \forall i \in R
$

**2. Neighborhood demand balance**

$ \displaystyle
\sum_{j \in D} y_{jk} + u_k = d_k \quad \forall k \in N
$

**3. Flow conservation at donation centers**

$ \displaystyle
\sum_{k \in N} y_{jk}
=
\sum_{i \in R} x_{ij}
\quad \forall j \in D
$

**4. Nonnegativity**

$ x_{ij},\; y_{jk},\; u_k \ge 0 $


In [60]:
# ================================
# 9. REDUCED COST OPTIMIZATION MODEL
# ================================

# Big-M penalty for unmet demand
# Here we pick M as the total demand so that leaving demand unmet is very expensive
M = sum(demand)

model_cost = Model(Gurobi.Optimizer)

# Decision variables:
# x[i,j] = flow of food from restaurant i to donation center j
# y[j,k] = flow of food from donation center j to neighborhood k
# u[k]   = unmet demand at neighborhood k
@variable(model_cost, x[1:R, 1:D] >= 0)
@variable(model_cost, y[1:D, 1:N] >= 0)
@variable(model_cost, u[1:N] >= 0)

# Objective:
# Minimize transportation cost + big-M penalty on unmet demand
@objective(model_cost, Min,
    sum(cij[i,j] * x[i,j] for i in 1:R, j in 1:D) +
    sum(cjk[j,k] * y[j,k] for j in 1:D, k in 1:N) +
    M * sum(u[k] for k in 1:N)
)

# Constraints:

# 1) Restaurant supply: cannot ship more than available surplus
@constraint(model_cost, [i in 1:R],
    sum(x[i,j] for j in 1:D) <= supply[i]
)

# 2) Neighborhood demand balance:
#    inflow from centers + unmet demand = demand
@constraint(model_cost, [k in 1:N],
    sum(y[j,k] for j in 1:D) + u[k] == demand[k]
)

# 3) Donation centers are pure transshipment nodes:
#    total inflow from restaurants = total outflow to neighborhoods
@constraint(model_cost, [j in 1:D],
    sum(y[j,k] for k in 1:N) == sum(x[i,j] for i in 1:R)
)

# ================================
# 10. SOLVE AND INSPECT
# ================================
optimize!(model_cost)

println("Termination status: ", termination_status(model_cost))
println("Objective value (total cost + penalty): ", objective_value(model_cost))

# Optional: extract solutions
x_opt = value.(x)
y_opt = value.(y)
u_opt = value.(u)

println("Total unmet demand: ", sum(u_opt))
println("Total shipped from restaurants: ", sum(x_opt))
println("Total received by neighborhoods: ", sum(y_opt))


Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13620H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1113 rows, 183903 columns and 367215 nonzeros
Model fingerprint: 0x45974fb3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-04, 2e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+02, 4e+06]
Presolve removed 433 rows and 87466 columns
Presolve time: 0.22s

Solved in 0 iterations and 0.22 seconds (0.03 work units)
Infeasible model

User-callback calls 20, time in user-callback 0.00 sec
INFEASIBLEn status: 


LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.